In [4]:
import os
import glob

from osgeo import gdal, osr
import geopandas as gpd

In [5]:
gdal.UseExceptions()
path_osgeo_utils = "C:\\NMBU\\Miniconda3\\envs\\deep_learning\\Lib\\site-packages\\GDAL-3.7.0-py3.11-win-amd64.egg-info\\scripts"

In [7]:
def tile_orthomosaic(
    input_ortho_path,
    output_path,
    footprints_path,
    tile_size,
    buffer_size,
):
    # read drone acquisition footprints
    footprints = gpd.read_file(footprints_path)
    # Get ortho name
    ortho_name = os.path.splitext(os.path.basename(input_ortho_path))[0]

    # create output dir
    output_tiles_dir = f"{output_path}\\{tile_size}m_{ortho_name}"
    if not os.path.exists(output_tiles_dir):
        print("Creating output folder..."+ output_tiles_dir)
        os.makedirs(output_tiles_dir)     

  # get raster metadata
  # Get pixel resolution (in meters) and tile size in pixels
    src_ds = gdal.Open(input_ortho_path) # reads in the orthomosaic
    _, xres, _, _, _, yres  = src_ds.GetGeoTransform() # get pixel size in meters
    print(f"Ortho resolution:{round(xres, 4)} m")
    # Get EPSG code
    proj = osr.SpatialReference(wkt=src_ds.GetProjection())
    EPSG_code= proj.GetAttrValue('AUTHORITY',1)
    print(f"EPSG code: {EPSG_code}")
    # get number of bands
    n_bands=src_ds.RasterCount
    print(f"Number of bands: {n_bands}")

    # Compute tile and buffer size in pixels
    tile_size_px = round(tile_size / abs(xres)) # calculate the tile size in pixels
    buffer_size_px = round(buffer_size / abs(xres)) # calculate the buffer size in pixels
    tileIndex_name = "tile_index" # define name for output tile index shapefile

    # Run gdal_retile.py using (can take some minutes) 
    command_retile = (
        "python " +
        path_osgeo_utils +
        "/gdal_retile.py -targetDir " +
        output_tiles_dir +
        " " +
        input_ortho_path +
        " -overlap "
        + str(buffer_size_px) +
        " -ps "
        + str(tile_size_px) +
        " " +
        str(tile_size_px) +
        " -of GTiff -tileIndex " +
        tileIndex_name +
        " -tileIndexField ID"
    )
    print(os.popen(command_retile).read())
    # cleanup tiles
    footprint_ortho = footprints[footprints['filename'] == ortho_name]
    footprint_ortho_UU = footprint_ortho.geometry.unary_union

    # Load tiles shapefile
    tiles = gpd.read_file(output_tiles_dir+ "/"+ortho_name+"_tile_index.shp")
    tiles= tiles.to_crs(EPSG_code)

    # Select all tiles that are within the boundary polygon
    tiles_in = tiles[tiles.geometry.within(footprint_ortho_UU)]

    # Select all tiles that are not within the boundary polygon
    tiles_out= tiles.loc[~tiles['ID'].isin(tiles_in['ID']) ]
    print(str(len(tiles_out))+" tiles to be deleted") 

    # delete tiles that are not within the footprint
    gtiffs_delete=[output_tiles_dir+ "/"+sub  for sub in tiles_out['ID']] 
    for f in gtiffs_delete:
        if os.path.exists(f):
            os.remove(f)

In [13]:
test_orthos = glob.glob("..\\data\\orthomosaics\\test_data\\*.tif")
output_path = "..\\data\\tiles\\test_data"
footprints_path = "..\\data\\map_data\\drone_acquisitions.geojson"

In [14]:
for i in test_orthos:
    print(f"Processing {i}")
    tile_orthomosaic(
        input_ortho_path=i,
        output_path=output_path,
        footprints_path=footprints_path,
        tile_size=10,
        buffer_size=1,
    )

Ortho resolution:0.0085 m
EPSG code: 25832
Number of bands: 4
0...10...20...30...40...50...60...70...80...90...100 - done.

1080 tiles to be deleted
Creating output folder.....\data\tiles\test_data\10m_galbyveien_20230504_sun
Ortho resolution:0.0108 m
EPSG code: 25832
Number of bands: 4
0...10...20...30...40...50...60...70...80...90...100 - done.

820 tiles to be deleted
Creating output folder.....\data\tiles\test_data\10m_krakstad_202304_sun
Ortho resolution:0.0077 m
EPSG code: 25832
Number of bands: 4
0...10...20...30...40...50...60...70...80...90...100 - done.

333 tiles to be deleted
Creating output folder.....\data\tiles\test_data\10m_ortho_hobol_042222_mavic_sun
Ortho resolution:0.0074 m
EPSG code: 25832
Number of bands: 4
0...10...20...30...40...50...60...70...80...90...100 - done.

261 tiles to be deleted
